In [ ]:
!pip install python-binance

In [ ]:
from binance import Client
import pandas as pd

In [ ]:
#Setting up client with api keys
client = Client(api_key, api_secret)

In [ ]:
#Checking API connection
client.get_account()

In [ ]:
#viewing historical data in a pandas dataframe
data = pd.DataFrame(client.get_historical_klines('BTCUSDT', '5m','1 h ago UTC' ))
data

In [ ]:
#Creating a function to get historical data, removing non essential columns, converting unix to datetime, converting objects to floats, naming columns
def getminutedata(symbol, interval, lookback):
    frame = pd.DataFrame(client.get_historical_klines(symbol, interval, lookback + ' h ago' ))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit='ms')
    frame = frame.astype(float)
    #frame['Volume'] = frame['Volume'] * frame['Close']
    return frame
    
    

In [ ]:
#checking our function
getminutedata('BTCUSDT', '5m', '5'  )

In [ ]:
dat.Open.plot()

In [ ]:
#Trading Strategy
def strategytest(symbol, qty, entered=False):
    df = getminutedata(symbol, '1m', '30m')
    cumulret = (df.Open.pct_change() +1).cumprod() - 1
    if not entered:
        if cumulret[-1] < -0.002:
            order = client.create_order(symbol=symbol, side='BUY', type='MARKET', quantity=qty)
            print(order)
            entered=True
        else:
            print('No Trade has been executed')
    if entered:
        while True:
            df = getminutedata(symbol, '1m', '30m')
            sincebuy = df.loc[df.index > pd.to_datetime(order['transactTime'], unit='ms')]
            if len(sincebuy) > 0:
                sincebuyret = (sincebuy.Open.pct_change() +1).cumprod() - 1
                if sincebuyret[-1] > 0.0015 or sincebuyret[-1] < -0.0015:
                    order = client.create_order(symbol=symbol, side='SELL', type='MARKET', quantity=qty)
                    print(order)
                    break
                

In [ ]:
strategytest('BTCUSDT', 0.001)

In [ ]:
strategytest('BTCUSDT', 0.001)

In [ ]:
strategytest('BTCUSDT', 0.001)

In [ ]:
strategytest('BTCUSDT', 0.001)